In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os.path
import pickle
import csv

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

In [13]:
#Read in data 
traindf = pd.read_csv('fb_kaggle_train.csv', delimiter=",", usecols=[1,2,3,4,5])
trainlabels = traindf['place_id']

traindf['hour'] = ( (traindf['time']+120)/60)%24+1 
traindf['weekday'] = (traindf['time']/1440)%7+1 
traindf['month'] = ( traindf['time'] /43800)%12+1 
traindf['year'] = (traindf['time']/525600)+1 
traindf['four_hour'] = (traindf['time']/240)%6+1
traindf['acc'] = np.log10(traindf['accuracy'])



In [14]:
weights = np.array([490.0, 980.0, 4.0, 3.1, 2.1, 10.0, 36]) #feature weights    X = grid_train[['x', 'y', 'hour', 'weekday', 'month', 'year', 'acc']].values * weights[g_id][:7]
traindf = traindf[['x', 'y', 'hour', 'weekday', 'month', 'year', 'acc']].values * weights


In [3]:
# save the classifier
if os.path.isfile('knn_classifier.pkl'):
    with open('knn_classifier.pkl', 'rb') as handle:
        knn = pickle.load(handle)
else:
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(traindf,trainlabels.values)
    
    with open('knn_classifier.pkl', 'wb') as handle:
        pickle.dump(knn, handle) 
    
#rows = np.random.choice(traindf.index.values, 2000000, replace=False)
#sampled_df = traindf.ix[rows]

#print(rows)
#print(sampled_df)
#knn.fit(sampled_df.iloc[:,0:4].values,np.ravel(sampled_df.iloc[:,4:].values))
#knn.fit(train_feat[:10000],train_labels[:10000])


In [2]:
testdf = pd.read_csv('fb_kaggle_test.csv', delimiter=",", usecols=[1,2,3,4])

testdf['hour'] = ( (testdf['time']+120)/60)%24+1 
testdf['weekday'] = (testdf['time']/1440)%7+1 
testdf['month'] = ( testdf['time'] /43800)%12+1 
testdf['year'] = (testdf['time']/525600)+1 
testdf['four_hour'] = (testdf['time']/240)%6+1
testdf['acc'] = np.log10(testdf['accuracy'])

weights = np.array([490.0, 980.0, 4.0, 3.1, 2.1, 10.0, 36]) #feature weights    X = grid_train[['x', 'y', 'hour', 'weekday', 'month', 'year', 'acc']].values * weights[g_id][:7]
testdf = testdf[['x', 'y', 'hour', 'weekday', 'month', 'year', 'acc']].values * weights

In [47]:
chunkSize = 1500
end = int(1+len(testdf)/chunkSize)
#prob = np.argsort(knn.predict_proba(testdf[:chunkSize]))
#top3 = [item[::-1][:3] for item in prob]
#del prob

for i in range(0,end):
    prob = np.argsort(knn.predict_proba(testdf[i*chunkSize:(i+1)*chunkSize]))
    top3 = [item[::-1][:3] for item in prob]
    preds = [knn.classes_[top] for top in top3]
    #print(top3)
    with open("new_kaggle_submission.csv", "a", newline='') as f:
        writer = csv.writer(f)
        writer.writerows(preds)
        f.close()
    del prob
    del top3
    del preds
    #pred = np.concatenate((pred,knn.predict(test[i*chunkSize:(i+1)*chunkSize])))
    #prob = np.concatenate((prob,knn.predict_proba(testdf[i*chunkSize:(i+1)*chunkSize])))

#This is how to extract the top three probabilities and extract predictions 
#print([prob[::-1][:3] for prob in probs])
#print(knn.classes_[prob[4][::-1][:3]])

In [9]:
preds = np.tile(pred,(3,1))
print(preds.shape)
index = np.arange(len(preds[0]))
print(index.shape)
preds = np.vstack((index,preds))
print(preds.shape)
#print(preds)
df = pd.DataFrame({"pred":preds[1]})

#print(dog)

df["pred"] = df.pred.map(str)


(3, 8607230)
(8607230,)
(4, 8607230)


In [10]:
#df["pred"] = df.pred + " " + df.pred + " " + df.pred
df.index.name = 'row_id'
df.columns = ['place_id']
print(df)
df.to_csv("fb_sample_submission.csv")


           place_id
row_id             
0        3071064812
1        3885244710
2        3071064812
3        3071064812
4        3885244710
5        3071064812
6        3885244710
7        3885244710
8        3071064812
9        3885244710
10       3071064812
11       3071064812
12       3885244710
13       3071064812
14       3071064812
15       3885244710
16       3885244710
17       3071064812
18       3885244710
19       3071064812
20       3071064812
21       3071064812
22       3885244710
23       3071064812
24       3071064812
25       3071064812
26       3071064812
27       3071064812
28       3071064812
29       3885244710
...             ...
8607200  3071064812
8607201  3071064812
8607202  3885244710
8607203  3071064812
8607204  3885244710
8607205  3885244710
8607206  3885244710
8607207  3071064812
8607208  3071064812
8607209  3071064812
8607210  3885244710
8607211  4015543882
8607212  3885244710
8607213  3071064812
8607214  3071064812
8607215  3885244710
8607216  3071064812
